# Context Utilization Measurement in RAG Systems

**Lesson 13 - Interactive Notebook**

This notebook demonstrates how to measure which retrieved context documents the LLM actually uses in its responses.

## Learning Objectives
- Measure semantic similarity between response and each context document
- Classify documents as USED (>0.7), PARTIAL (0.4-0.7), or IGNORED (<0.4)
- Identify which retrieved documents contribute to the response
- Detect retrieval waste (unused relevant documents)

## Execution Time & Cost
- **DEMO Mode**: ~2 minutes, $0.03 (20 test cases)
- **FULL Mode**: ~6 minutes, $1.50-2.50 (200 test cases)

**⚠️ COST WARNING**: Running FULL mode will use OpenAI API credits for embeddings.

In [ ]:
# Setup and imports
import json
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from backend.rag_generation_eval import ContextUtilizationScorer
from datetime import datetime

# Configure visualization
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports successful")

In [ ]:
# Configuration: DEMO or FULL mode
MODE = "DEMO"  # Change to "FULL" for comprehensive evaluation

N_SAMPLES = 20 if MODE == "DEMO" else 200
ESTIMATED_COST = "$0.03" if MODE == "DEMO" else "$1.50-2.50"
ESTIMATED_TIME = "2 minutes" if MODE == "DEMO" else "6 minutes"

print(f"🔧 Mode: {MODE}")
print(f"📊 Samples: {N_SAMPLES}")
print(f"💰 Estimated Cost: {ESTIMATED_COST}")
print(f"⏱️  Estimated Time: {ESTIMATED_TIME}")

## Step 1: Load RAG Evaluation Test Suite

We'll use the same 500-case test suite, but focus on multi-document context cases.

In [ ]:
# Load test suite
test_suite_path = Path("data/rag_evaluation_suite.json")

if not test_suite_path.exists():
    raise FileNotFoundError(f"Test suite not found at {test_suite_path}")

with open(test_suite_path, "r", encoding="utf-8") as f:
    test_suite = json.load(f)

all_test_cases = test_suite["test_cases"]

# Sample based on mode
import random
random.seed(42)  # Reproducibility
test_cases = random.sample(all_test_cases, min(N_SAMPLES, len(all_test_cases)))

print(f"✅ Loaded {len(all_test_cases)} test cases from suite")
print(f"📋 Selected {len(test_cases)} test cases for {MODE} mode")

## Step 2: Initialize Context Utilization Scorer

The `ContextUtilizationScorer` measures semantic similarity between response and each context document.

In [ ]:
# Initialize scorer
scorer = ContextUtilizationScorer()

print("✅ ContextUtilizationScorer initialized")
print("\nScorer Methods:")
print("  - measure_utilization(): Calculate similarity for each context doc")
print("  - classify_usage(): Classify as USED/PARTIAL/IGNORED")
print("\nThresholds:")
print("  - USED: Similarity > 0.7")
print("  - PARTIAL: Similarity 0.4 - 0.7")
print("  - IGNORED: Similarity < 0.4")

## Step 3: Example - Single Case Utilization

Let's see how context utilization works on a single example with multiple context documents.

In [ ]:
# Example with multiple context documents
example_case = {
    "query": "What is Krishna's teaching to Arjuna?",
    "context": [
        "Krishna teaches Arjuna about duty and dharma in the Bhagavad Gita.",  # USED
        "The Mahabharata is an ancient Indian epic about a great war.",  # PARTIAL
        "Chocolate chip cookies require flour, sugar, butter, and chocolate chips.",  # IGNORED
    ],
    "response": "Krishna teaches Arjuna about his duty and the importance of dharma."
}

print("📖 Example Case: Context Utilization\n")
print(f"Query: {example_case['query']}")
print(f"\nResponse: {example_case['response']}")
print(f"\nContext Documents ({len(example_case['context'])}):\n")
for i, doc in enumerate(example_case['context']):
    print(f"{i + 1}. {doc}")

# Measure utilization
utilization = scorer.measure_utilization(example_case['response'], example_case['context'])

print(f"\n📊 Utilization Scores:")
for doc_idx, similarity in utilization.items():
    usage = scorer.classify_usage(similarity)
    emoji = "🟩" if usage == "USED" else "🟨" if usage == "PARTIAL" else "🟥"
    print(f"   Doc {doc_idx + 1}: {similarity:.3f} {emoji} {usage}")

print("\n✅ As expected:")
print("   - Doc 1 (Krishna teaching) is USED")
print("   - Doc 2 (Mahabharata context) is PARTIAL")
print("   - Doc 3 (cookie recipe) is IGNORED")

## Step 4: Batch Evaluation on Test Suite

Evaluate context utilization across all test cases.

In [ ]:
# Batch evaluation
from tqdm import tqdm

results = []
errors = 0

print(f"🔄 Measuring context utilization for {len(test_cases)} test cases...\n")

for test_case in tqdm(test_cases, desc="Context Utilization"):
    try:
        # Extract fields
        answer = test_case.get("answer", "")
        context = test_case.get("context", [])
        ground_truth_usage = test_case.get("labels", {}).get("context_utilization", None)
        
        # Measure utilization
        utilization = scorer.measure_utilization(answer, context)
        
        # Classify each document
        doc_classifications = {}
        for doc_idx, similarity in utilization.items():
            usage = scorer.classify_usage(similarity)
            doc_classifications[doc_idx] = {
                "similarity": similarity,
                "usage": usage,
            }
        
        # Aggregate: what's the overall utilization?
        # Classify as USED if at least 1 doc is USED, PARTIAL if at least 1 PARTIAL, else IGNORED
        usages = [doc["usage"] for doc in doc_classifications.values()]
        if "USED" in usages:
            overall_usage = "USED"
        elif "PARTIAL" in usages:
            overall_usage = "PARTIAL"
        else:
            overall_usage = "IGNORED"
        
        # Store result
        results.append({
            "id": test_case.get("id"),
            "source": test_case.get("source"),
            "query": test_case.get("query"),
            "answer": answer,
            "context_count": len(context),
            "utilization": utilization,
            "doc_classifications": doc_classifications,
            "overall_usage": overall_usage,
            "ground_truth_usage": ground_truth_usage,
            "correct_classification": overall_usage == ground_truth_usage if ground_truth_usage else None,
        })
        
    except Exception as e:
        errors += 1
        print(f"⚠️  Error on test case {test_case.get('id')}: {e}")

print(f"\n✅ Evaluation complete!")
print(f"   Total cases: {len(results)}")
print(f"   Errors: {errors}")

## Step 5: Analyze Results

Compute metrics and analyze context utilization patterns.

In [ ]:
# Calculate metrics
total_cases = len(results)

# Overall utilization distribution
usage_counts = {"USED": 0, "PARTIAL": 0, "IGNORED": 0}
for result in results:
    usage_counts[result["overall_usage"]] += 1

# Accuracy (if ground truth available)
cases_with_ground_truth = [r for r in results if r["correct_classification"] is not None]
correct_classifications = sum(1 for r in cases_with_ground_truth if r["correct_classification"])
accuracy = (
    correct_classifications / len(cases_with_ground_truth)
    if cases_with_ground_truth
    else 0.0
)

# Average utilization per document
all_similarities = []
for result in results:
    for doc_idx, doc_info in result["doc_classifications"].items():
        all_similarities.append(doc_info["similarity"])

avg_similarity = sum(all_similarities) / len(all_similarities) if all_similarities else 0.0

# By source
source_breakdown = {}
for result in results:
    source = result["source"]
    if source not in source_breakdown:
        source_breakdown[source] = {"USED": 0, "PARTIAL": 0, "IGNORED": 0, "total": 0}
    source_breakdown[source][result["overall_usage"]] += 1
    source_breakdown[source]["total"] += 1

# Print metrics
print("📊 CONTEXT UTILIZATION RESULTS")
print("=" * 50)
print(f"\n📈 Overall Metrics:")
print(f"   Total test cases: {total_cases}")
print(f"   USED: {usage_counts['USED']} ({usage_counts['USED'] / total_cases * 100:.1f}%)")
print(f"   PARTIAL: {usage_counts['PARTIAL']} ({usage_counts['PARTIAL'] / total_cases * 100:.1f}%)")
print(f"   IGNORED: {usage_counts['IGNORED']} ({usage_counts['IGNORED'] / total_cases * 100:.1f}%)")
print(f"   Avg Similarity: {avg_similarity:.3f}")
print(f"   Classification Accuracy: {accuracy * 100:.1f}% (vs ground truth)")

print(f"\n📂 Breakdown by Source:")
for source, stats in source_breakdown.items():
    print(f"\n   {source}:")
    print(f"      USED: {stats['USED']}/{stats['total']} ({stats['USED'] / stats['total'] * 100:.1f}%)")
    print(f"      PARTIAL: {stats['PARTIAL']}/{stats['total']} ({stats['PARTIAL'] / stats['total'] * 100:.1f}%)")
    print(f"      IGNORED: {stats['IGNORED']}/{stats['total']} ({stats['IGNORED'] / stats['total'] * 100:.1f}%)")

## Step 6: Visualize Utilization Heatmap

Create a heatmap showing which documents are used across test cases.

In [ ]:
import numpy as np

# Prepare heatmap data (first 20 cases for readability)
n_display = min(20, len(results))
max_docs = max(r["context_count"] for r in results[:n_display])

heatmap_data = np.zeros((n_display, max_docs))
for i, result in enumerate(results[:n_display]):
    for doc_idx, doc_info in result["doc_classifications"].items():
        heatmap_data[i][doc_idx] = doc_info["similarity"]

# Create heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    heatmap_data,
    cmap="RdYlGn",
    vmin=0,
    vmax=1,
    cbar_kws={"label": "Similarity Score"},
    xticklabels=[f"Doc {i+1}" for i in range(max_docs)],
    yticklabels=[f"Case {i+1}" for i in range(n_display)],
)

plt.title("Context Utilization Heatmap\n(Green = USED, Yellow = PARTIAL, Red = IGNORED)", fontsize=14)
plt.xlabel("Context Document Index")
plt.ylabel("Test Case Index")
plt.tight_layout()
plt.show()

print("\n📊 Heatmap Legend:")
print("   🟩 Green (>0.7): USED - LLM actively referenced document")
print("   🟨 Yellow (0.4-0.7): PARTIAL - LLM partially used document")
print("   🟥 Red (<0.4): IGNORED - LLM did not use document")

## Step 7: Identify Retrieval Waste

Find cases where relevant documents were retrieved but not used.

In [ ]:
# Find cases with IGNORED documents
wasted_retrievals = []
for result in results:
    ignored_count = sum(1 for doc in result["doc_classifications"].values() if doc["usage"] == "IGNORED")
    if ignored_count > 0:
        wasted_retrievals.append({
            "id": result["id"],
            "source": result["source"],
            "ignored_count": ignored_count,
            "total_docs": result["context_count"],
            "waste_rate": ignored_count / result["context_count"],
        })

# Sort by waste rate
wasted_retrievals.sort(key=lambda x: x["waste_rate"], reverse=True)

print(f"⚠️  Retrieval Waste Analysis: {len(wasted_retrievals)} cases with ignored documents\n")

# Show top 5 wasted cases
for i, case in enumerate(wasted_retrievals[:5]):
    print(f"{i + 1}. {case['id']} ({case['source']})")
    print(f"   Ignored: {case['ignored_count']}/{case['total_docs']} ({case['waste_rate'] * 100:.1f}%)\n")

# Overall waste rate
total_docs = sum(r["context_count"] for r in results)
total_ignored = sum(
    sum(1 for doc in r["doc_classifications"].values() if doc["usage"] == "IGNORED")
    for r in results
)
overall_waste_rate = total_ignored / total_docs if total_docs > 0 else 0.0

print(f"📊 Overall Retrieval Waste Rate: {overall_waste_rate * 100:.1f}%")
print(f"   (Out of {total_docs} retrieved documents, {total_ignored} were ignored)")

## Step 8: Export Results for Dashboard

Save results to JSON for integration with evaluation dashboard.

In [ ]:
# Prepare export data
export_data = {
    "metadata": {
        "lesson": "Lesson 13 - RAG Generation & Attribution",
        "notebook": "context_utilization.ipynb",
        "mode": MODE,
        "timestamp": datetime.now().isoformat(),
        "test_cases": len(results),
    },
    "metrics": {
        "usage_distribution": usage_counts,
        "avg_similarity": avg_similarity,
        "classification_accuracy": accuracy,
        "retrieval_waste_rate": overall_waste_rate,
        "source_breakdown": source_breakdown,
    },
    "results": results,
}

# Save to results directory
output_path = Path("results/context_utilization_results.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

print(f"✅ Results exported to {output_path}")
print(f"📊 File size: {output_path.stat().st_size / 1024:.1f} KB")
print(f"\n🎯 Ready for dashboard integration!")

## Summary

**What we learned:**
1. How to measure semantic similarity between response and context
2. How to classify documents as USED/PARTIAL/IGNORED
3. How to identify retrieval waste (unused documents)
4. How to visualize context utilization patterns

**Key Insights:**
- Not all retrieved documents are used by the LLM
- Gita Q&A tends to have higher context utilization (focused queries)
- Recipe responses may partially use multiple documents
- Retrieval waste indicates opportunity for reranking
- Low utilization may signal irrelevant retrieval or LLM ignoring context

**Next Steps:**
- Read [End-to-End RAG Evaluation Tutorial](./end_to_end_rag_eval.md)
- Try [Attribution Detection Notebook](./attribution_detection.ipynb)
- Explore [RAG Failure Taxonomy Diagram](./diagrams/rag_failure_taxonomy.mmd)